In [1]:
import os
import pandas as pd
import sqlalchemy as db
from dotenv import load_dotenv

In [2]:
load_dotenv()

# config and credentials
server = os.getenv('server')
database = os.getenv('database')
username = os.getenv('username')
password = os.getenv('password')

# connection
engine = db.create_engine(
    'mssql://{}:{}@{}/{}?driver=ODBC+Driver+18+for+SQL+Server'.format(
        username, password, server, database
    )
)

# establish connection
connection = engine.connect()

### CSAD Bonds

In [3]:
query = '''
    SELECT
        Date,
        Rm,
        ABS(Rm) AS AbsoluteRm,
        POWER(Rm, 2) AS SquaredRm,
        Sum / Count AS Csad
    FROM (
        SELECT
            Date,
            Rm,
            ABS(SUM(RetEom) - Rm) AS Sum,
            COUNT(DISTINCT Cusip) AS Count
        FROM (
            SELECT
                A.TrdExctnDtEOM AS Date,
                A.CusipId AS Cusip,
                A.R AS RetEom,
                B.Rm
            FROM
                BondReturns_customerOnly A
            INNER JOIN (
                SELECT
                    TrdExctnDtEOM AS Date,
                    SUM(R * TD_volume) / SUM(TD_volume) AS Rm
                FROM
                    BondReturns_customerOnly
                GROUP BY
                    TrdExctnDtEOM
            ) B ON A.TrdExctnDtEOM = B.Date
        ) A
        GROUP BY
            Date,
            Rm
    ) B
    WHERE
	    Rm IS NOT NULL
    ORDER BY
        Date
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/csad-bonds.csv', index=False)

### Institutional

In [4]:
query = '''
    SELECT
        Date,
        Rm,
        ABS(Rm) AS AbsoluteRm,
        POWER(Rm, 2) AS SquaredRm,
        Sum / Count AS Csad
    FROM (
        SELECT
            Date,
            Rm,
            ABS(SUM(RetEom) - Rm) AS Sum,
            COUNT(DISTINCT Cusip) AS Count
        FROM (
            SELECT
                A.TrdExctnDtEOM AS Date,
                A.CusipId AS Cusip,
                A.R AS RetEom,
                B.Rm
            FROM
                BondReturns_customerOnly_institutional A
            INNER JOIN (
                SELECT
                    TrdExctnDtEOM AS Date,
                    SUM(R * TD_volume) / SUM(TD_volume) AS Rm
                FROM
                    BondReturns_customerOnly_institutional
                GROUP BY
                    TrdExctnDtEOM
            ) B ON A.TrdExctnDtEOM = B.Date
        ) A
        GROUP BY
            Date,
            Rm
    ) B
    WHERE
	    Rm IS NOT NULL
    ORDER BY
        Date
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/csad-bonds-institutional.csv', index=False)

### Retail

In [5]:
query = '''
    SELECT
        Date,
        Rm,
        ABS(Rm) AS AbsoluteRm,
        POWER(Rm, 2) AS SquaredRm,
        Sum / Count AS Csad
    FROM (
        SELECT
            Date,
            Rm,
            ABS(SUM(RetEom) - Rm) AS Sum,
            COUNT(DISTINCT Cusip) AS Count
        FROM (
            SELECT
                A.TrdExctnDtEOM AS Date,
                A.CusipId AS Cusip,
                A.R AS RetEom,
                B.Rm
            FROM
                BondReturns_customerOnly_retail A
            INNER JOIN (
                SELECT
                    TrdExctnDtEOM AS Date,
                    SUM(R * TD_volume) / SUM(TD_volume) AS Rm
                FROM
                    BondReturns_customerOnly_retail
                GROUP BY
                    TrdExctnDtEOM
            ) B ON A.TrdExctnDtEOM = B.Date
        ) A
        GROUP BY
            Date,
            Rm
    ) B
    WHERE
	    Rm IS NOT NULL
    ORDER BY
        Date
'''

# read sql
df = pd.read_sql_query(query, connection)
df.to_csv('source/csad-bonds-retail.csv', index=False)